In [134]:
import importlib
import extraction_functions_praat
import features
importlib.reload(extraction_functions_praat)
importlib.reload(features)

<module 'features' from '/home/dene/rp2/features.py'>

In [135]:
from paths import *
from features import *
import pandas as pd
from extraction_functions_praat import (
    PP_f0_mean, PP_f0_median, PP_f0_sd,
    PP_f0_mean_murton, PP_f0_median_murton, PP_f0_sd_murton, 
    PP_jitter, PP_jitter_murton,
    PP_lh_ratio, PP_LH_ratio_murton,
    PP_CPP_mean_murton, PP_CPP_median_murton, PP_CPP_sd_murton,
    PP_duration_with_pauses, PP_duration_without_pauses,
    PP_harmonics_to_noise, PP_harmonics_to_noise_murton,
    PP_shimmer, PP_shimmer_murton,
    PP_MFCC
)

In [136]:
def extract_features_VOW(audio_file, selected_features, segment_length, f0_min, f0_max):
    extracted_features = {}
    
    if 'PP_F0' in selected_features:
        extracted_features['PP_F0'] = PP_f0_mean(audio_file, f0_min=f0_min, f0_max=f0_max)
    if 'PP_F0_M' in selected_features:
        extracted_features['PP_F0_M'] = PP_f0_mean_murton(audio_file, f0_min=f0_min, f0_max=f0_max)
    if 'PP_F0_SD' in selected_features:
        extracted_features['PP_F0_SD'] = PP_f0_sd(audio_file, f0_min=f0_min, f0_max=f0_max)
    if 'PP_F0_SD_M' in selected_features:
        extracted_features['PP_F0_SD_M'] = PP_f0_sd_murton(audio_file, f0_min=f0_min, f0_max=f0_max)

    if 'PP_JIT' in selected_features:
        jitter_values = PP_jitter(audio_file, f0_min=f0_min, f0_max=f0_max) 
        for feature in jitter_feature_selection:
            if feature in jitter_feature_indices:
                feature_idx = jitter_feature_indices[feature]            
                extracted_features[f'PP_JIT_{feature}'] = jitter_values[feature_idx]  
    if 'PP_JIT_M' in selected_features:
        jitter_values = PP_jitter_murton(audio_file, segment_length=segment_length, f0_min=f0_min, f0_max=f0_max) 
        for feature in jitter_feature_selection:
            if feature in jitter_feature_indices:
                feature_idx = jitter_feature_indices[feature]            
                extracted_features[f'PP_JIT_M_{feature}'] = jitter_values[feature_idx]    

    if 'PP_LHR' in selected_features:
        extracted_features['PP_LHR'] = PP_lh_ratio(audio_file)
    if 'PP_LHR_M' in selected_features:
        extracted_features['PP_LHR_M'] = PP_LH_ratio_murton(audio_file, segment_length=segment_length)

    if 'PP_CPP_M' in selected_features:
        extracted_features['PP_CPP_M'] = PP_CPP_mean_murton(audio_file)
    if 'PP_CPP_M2' in selected_features:
        extracted_features['PP_CPP_M2'] = PP_CPP_median_murton(audio_file)
    if 'PP_CPP_SD_M' in selected_features:
        extracted_features['PP_CPP_SD_M'] = PP_CPP_sd_murton(audio_file)

    if 'PP_HNR' in selected_features:
        extracted_features['PP_HNR'] = PP_harmonics_to_noise(audio_file)
    if 'PP_HNR_M' in selected_features:
        extracted_features['PP_HNR_M'] = PP_harmonics_to_noise_murton(audio_file, segment_length=segment_length)

    if 'PP_SHI' in selected_features:
        shimmer_values = PP_shimmer(audio_file, f0_min=f0_min, f0_max=f0_max) 
        for feature in shimmer_feature_selection:
            if feature in shimmer_feature_indices:
                feature_idx = shimmer_feature_indices[feature]            
                extracted_features[f'PP_SHI_{feature}'] = shimmer_values[feature_idx]
    if 'PP_SHI_M' in selected_features:
        shimmer_values = PP_shimmer_murton(audio_file, segment_length=segment_length, f0_min=f0_min, f0_max=f0_max) 
        for feature in shimmer_feature_selection:
            if feature in shimmer_feature_indices:
                feature_idx = shimmer_feature_indices[feature]            
                extracted_features[f'PP_SHI_M_{feature}'] = shimmer_values[feature_idx]
            
    return extracted_features


def extract_features_MPT(audio_file, selected_features, silence_threshold, min_silence_duration):
    extracted_features = {}
    
    if 'PP_DUR_WP' in selected_features:
        extracted_features['PP_DUR_WP'] = PP_duration_with_pauses(audio_file, silence_threshold)
    if 'PP_DUR_WOP' in selected_features:
        extracted_features['PP_DUR_WOP'] = PP_duration_without_pauses(audio_file, silence_threshold, min_silence_duration)
            
    return extracted_features


def extract_features_SEN(audio_file, selected_features, f0_min, f0_max, silence_threshold, min_silence_duration, num_coefficients):
    extracted_features = {}
    
    if 'PP_F0' in selected_features:
        extracted_features['PP_F0'] = PP_f0_median(audio_file, f0_min=f0_min, f0_max=f0_max)    
    if 'PP_F0_M' in selected_features:
        extracted_features['PP_F0_M'] = PP_f0_median_murton(audio_file, f0_min=f0_min, f0_max=f0_max)  
    if 'PP_F0_SD' in selected_features:
        extracted_features['PP_F0_SD'] = PP_f0_sd(audio_file, f0_min=f0_min, f0_max=f0_max)
    if 'PP_F0_SD_M' in selected_features:
        extracted_features['PP_F0_SD_M'] = PP_f0_sd_murton(audio_file, f0_min=f0_min, f0_max=f0_max)
        
    if 'PP_CPP_M' in selected_features:
        extracted_features['PP_CPP_M'] = PP_CPP_mean_murton(audio_file)
    if 'PP_CPP_M2' in selected_features:
        extracted_features['PP_CPP_M2'] = PP_CPP_median_murton(audio_file)
    if 'PP_CPP_SD_M' in selected_features:
        extracted_features['PP_CPP_SD_M'] = PP_CPP_sd_murton(audio_file)
        
    if 'PP_DUR_WP' in selected_features:
        extracted_features['PP_DUR_WP'] = PP_duration_with_pauses(audio_file, silence_threshold)
    if 'PP_DUR_WOP' in selected_features:
        extracted_features['PP_DUR_WOP'] = PP_duration_without_pauses(audio_file, silence_threshold, min_silence_duration)
        
    if 'PP_MFC' in selected_features:
        mfc_values = PP_MFCC(audio_file, num_coefficients=num_coefficients)
        for i in range(num_coefficients):
            for feature in mfc_feature_selection:
                if feature in mfc_feature_indices:
                    feature_idx = mfc_feature_indices[feature]
                    extracted_features[f'PP_MFC_{i+1}_{feature}'] = mfc_values[i+(num_coefficients*feature_idx)]

    return extracted_features

In [137]:
def process_audio_files_VOW(directory, selected_features, segment_length, f0_min, f0_max):
    patient_dfs = {}

    files = [file for d in directory for file in d.rglob('*') if file.is_file()]

    for file in files:
        filename = file.stem.replace("_pre", "")
        parts = filename.split("_")
        if len(parts) != 4:
            print(f"Unexpected named audio file: {file}")
            continue

        patient_id, day, exercise, take_letter = parts

        features = {}
        file_path = str(file)
        
        if exercise == 'VOW':
            if directory == best_segments_dir:
                if file_path[-1].isdigit(): 
                    continue ## we do not analyze the best segment of the entire vowel exercise, only of the best segments
                if file_path[-1].isalpha():
                    features = extract_features_VOW(
                        file_path, selected_features, 
                        f0_min, f0_max)
            else:
                features = extract_features_VOW(
                    file_path, selected_features, 
                    segment_length, 
                    f0_min, f0_max)
        
        if features:    
            df_entry = {'day': int(day), **features}       
            df_key = (patient_id, take_letter, exercise)
            if df_key not in patient_dfs:
                patient_dfs[df_key] = pd.DataFrame(columns=['day'] + list(features.keys()))
            patient_dfs[df_key] = pd.concat([patient_dfs[df_key], pd.DataFrame([df_entry])], ignore_index=True)
        
        for key, df in patient_dfs.items():
            patient_dfs[key] = df.sort_values(by='day', ascending=True).reset_index(drop=True)

        for (patient_id, take_letter, exercise), df in patient_dfs.items():
            file_name = f"{patient_id}_{exercise}_{take_letter}.csv"
            file_path = features_dir / exercise / patient_id / file_name

            df.to_csv(file_path, index=False)


def process_audio_files_MPT(directory, selected_features, silence_threshold, min_silence_duration):
    patient_dfs = {}

    files = [file for d in directory for file in d.rglob('*') if file.is_file()]

    for file in files:
        filename = file.stem.replace("_pre", "")
        parts = filename.split("_")
        if len(parts) != 4:
            print(f"Unexpected named audio file: {file}")
            continue

        patient_id, day, exercise, take_letter = parts

        features = {}
        file_path = str(file)
        
        if exercise == 'MPT':
            features = extract_features_MPT(
                file_path, selected_features, 
                silence_threshold, min_silence_duration)
            
        if features:            
            df_entry = {'day': int(day), **features}    
            df_key = (patient_id, take_letter, exercise)
            if df_key not in patient_dfs:
                patient_dfs[df_key] = pd.DataFrame(columns=['day'] + list(features.keys()))
            patient_dfs[df_key] = pd.concat([patient_dfs[df_key], pd.DataFrame([df_entry])], ignore_index=True)
            
        for key, df in patient_dfs.items():
            patient_dfs[key] = df.sort_values(by='day', ascending=True).reset_index(drop=True)

        for (patient_id, take_letter, exercise), df in patient_dfs.items():
            file_name = f"{patient_id}_{exercise}_{take_letter}.csv"
            file_path = features_dir / exercise / patient_id / file_name

            df.to_csv(file_path, index=False)


def process_audio_files_SEN(directory, selected_features, f0_min, f0_max, silence_threshold, min_silence_duration, num_coefficients):
    patient_dfs = {}

    files = [file for d in directory for file in d.rglob('*') if file.is_file()]

    for file in files:
        filename = file.stem.replace("_pre", "")
        parts = filename.split("_")
        if len(parts) != 4:
            print(f"Unexpected named audio file: {file}")
            continue

        patient_id, day, exercise, take_letter = parts
        
        features = {}
        file_path = str(file)
        
        if exercise == 'SEN':
            features = extract_features_SEN(
                file_path, selected_features, 
                f0_min, f0_max, 
                silence_threshold, min_silence_duration, 
                num_coefficients)
            
        if features:
            df_entry = {'day': int(day), **features}    
            df_key = (patient_id, take_letter, exercise)
            if df_key not in patient_dfs:
                patient_dfs[df_key] = pd.DataFrame(columns=['day'] + list(features.keys()))
            patient_dfs[df_key] = pd.concat([patient_dfs[df_key], pd.DataFrame([df_entry])], ignore_index=True)
        
        for key, df in patient_dfs.items():
            patient_dfs[key] = df.sort_values(by='day', ascending=True).reset_index(drop=True)
            
        for (patient_id, take_letter, exercise), df in patient_dfs.items():
            file_name = f"{patient_id}_{exercise}_{take_letter}.csv"
            file_path = features_dir / exercise / patient_id / file_name

            df.to_csv(file_path, index=False)

In [138]:
dir_to_run_vowels = [
    audio_dir,
    segments_dir,
    # best_segments_dir, ## will not use the best segments for now, since I might use the middle of a vowel segment (Murton 2023)
]
dir_to_run_mpt = [audio_dir, segments_dir]
dir_to_run_sentences = [audio_dir, segments_dir]

process_audio_files_VOW(
    directory=dir_to_run_vowels,
    selected_features=selected_features_dict,
    segment_length=1.0,
    f0_min=60, f0_max=300)

# process_audio_files_MPT(
#     directory=dir_to_run_mpt,
#     selected_features=selected_features_dict,
#     silence_threshold=50, min_silence_duration=0.5)

# process_audio_files_SEN(
#     directory=dir_to_run_sentences,
#     selected_features=selected_features_dict,
#     f0_min=60, f0_max=300,
#     silence_threshold=50, min_silence_duration=0.5,
#     num_coefficients=12)

PraatError: Command "Get shimmer (local)" not available for given objects.